In [1]:
# Read the source file and store chunks in an array
import os

def loadFAQs(directory_path):
   faqs = {}

   for filename in os.listdir(directory_path):
      if filename.endswith(".txt"):  # assuming FAQs are in .txt files
         file_path = os.path.join(directory_path, filename)

         with open(file_path) as f:
            raw_faq = f.read()

         filename_without_ext = os.path.splitext(filename)[0]  # remove .txt extension
         faqs[filename_without_ext] = [text.strip() for text in raw_faq.split('=====')]

   return faqs


In [2]:
# Store data from txt-docs
faqs = loadFAQs('./txt-docs')
faqs

{'faq': ['Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'What is Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier allows you to sign up for an Oracle Cloud account which provides a number of Always Free services and a Free Trial with US$300 of free credit to use on all eligible Oracle Cloud Infrastructure services for up to 30 days. The Always Free services are available for an unlimited period of time. The Free Trial services may be used until your US$300 of free credits are consumed or the 30 days has expired, whichever comes first.',
  'Who should use Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier services are for everyone. Whether you’re a developer building and testing applications, a startup founder creating new systems with the intention of scaling later, an enterprise looking to test thi

In [3]:
docs = [{'text': filename + ' | ' + section, 'path': filename} for filename, sections in faqs.items() for section in sections]

# Sample the resulting data
docs[:2]

[{'text': 'faq | Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'path': 'faq'},
 {'text': 'faq | What is Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier allows you to sign up for an Oracle Cloud account which provides a number of Always Free services and a Free Trial with US$300 of free credit to use on all eligible Oracle Cloud Infrastructure services for up to 30 days. The Always Free services are available for an unlimited period of time. The Free Trial services may be used until your US$300 of free credits are consumed or the 30 days has expired, whichever comes first.',
  'path': 'faq'}]

In [4]:
# Connect to the Oracle Database 23ai
un = "vector"
pw = "vector"
cs = "localhost/FREEPDB1"

import oracledb

connection = oracledb.connect(user=un, password=pw, dsn=cs)

In [5]:
table_name = 'faqs'

with connection.cursor() as cursor:
    # Create the table
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id NUMBER PRIMARY KEY,
            payload CLOB CHECK (payload IS JSON),
            vector VECTOR
        )"""
    try:
        cursor.execute(create_table_sql)
    except oracledb.DatabaseError as e:
        raise

    connection.autocommit = True

In [6]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L12-v2')

/home/opc/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
import array

# Define a list to store the data
data = [
   {"id": idx, "vector_source": row['text'], "payload": row} 
   for idx, row in enumerate(docs)
]

# Collect all texts for batch encoding
texts = [f"{row['vector_source']}" for row in data]

# Encode all texts in a batch
embeddings = encoder.encode(texts, batch_size=32, show_progress_bar=True)

# Assign the embeddings back to your data structure
for row, embedding in zip(data, embeddings):
   row['vector'] = array.array("f", embedding)

Batches: 100%|██████████| 7/7 [00:05<00:00,  1.37it/s]


In [8]:
# Insert the chunks + vectors in the database
import json

with connection.cursor() as cursor:
    # Truncate the table
    cursor.execute(f"truncate table {table_name}")

    prepared_data = [(row['id'], json.dumps(row['payload']), row['vector']) for row in data]

    # Insert the data
    cursor.executemany(
        f"""INSERT INTO {table_name} (id, payload, vector)
        VALUES (:1, :2, :3)""",
        prepared_data
    )

    connection.commit()

In [9]:
with connection.cursor() as cursor:
    # Define the query to select all rows from a table
    query = f"SELECT * FROM {table_name}"

    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the rows
    for row in rows[:5]:
        print(row)

(2, {'text': 'faq | Who should use Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier services are for everyone. Whether you’re a developer building and testing applications, a startup founder creating new systems with the intention of scaling later, an enterprise looking to test things before moving to cloud, a student wanting to learn, or an academic developing curriculum in the cloud, Oracle Cloud Free Tier enables you to learn, explore, build and test for free.', 'path': 'faq'}, array('f', [0.0475485734641552, -0.02257898636162281, 0.014575185254216194, -0.021478625014424324, 0.012853561900556087, -0.013652878813445568, -0.07855578511953354, 0.05885819345712662, -0.0038798199966549873, 0.002898386213928461, 0.02300194837152958, 0.021663479506969452, -0.014754838310182095, -0.05274977907538414, 0.029338177293539047, -0.059429824352264404, 0.02058180421590805, 0.0016598980873823166, 0.004078279715031385, -0.05929173156619072, -0.049143239855766296, -0.03091178461909294, -0.02429285086